In [118]:
import torch
import os

from torchvision import datasets

import numpy as np
from glob import glob

from PIL import Image

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

import torchvision.transforms as transforms

import torch.nn as nn
import torch.nn.functional as F

import gc

In [119]:
files = np.array(glob("./train_images_100/*"))

In [120]:
#Preprocessing (will add test images later) (Augmentation will be removed later once run with entire dataset)
preprocess = {
    'train_images_100': transforms.Compose([
        transforms.RandomResizedCrop(224), #Tensor size
        transforms.RandomHorizontalFlip(), #Augment the data to work with a small data set size (100 right now)
        transforms.RandomRotation(15), #Augment the data
        transforms.ToTensor(),
        transforms.Normalize(mean = [0.5, 0.5, 0.5], std = [0.5, 0.5, 0.5])
    ]),
    
    'test_images_100': transforms.Compose([
        transforms.Resize(256), #Resize to image size
        transforms.CenterCrop(224), #Tensor size
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std = [0.5, 0.5, 0.5])
        
    ])
    
}

image_path = "./"

#Load the images
images = {i: datasets.ImageFolder(os.path.join(image_path, i), preprocess[i])
          for i in ['train_images_100', 'test_images_100']}
loader = {x: torch.utils.data.DataLoader(images[x], batch_size = 20, shuffle = True, num_workers = 0) 
                for x in ['train_images_100', 'test_images_100']}

In [121]:
#Making the CNN
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        
        #Layer 1
        self.layer1 =  nn.Sequential(
            nn.Conv2d(3, 16, 3, padding = 1),
            nn.BatchNorm2d(16, 3),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        #Layer 2
        self.layer2 =  nn.Sequential(
            nn.Conv2d(16, 32, 3, padding = 1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2)
        )
        
        #Layer 3
        self.layer3 =  nn.Sequential(
            nn.Conv2d(32, 64, 3, padding = 1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2)
        )
        
        #Layer 4
        self.layer4 =  nn.Sequential(
            nn.Conv2d(64, 128, 3, padding = 1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2)
        )
        
        #Layer 5
        self.layer5 =  nn.Sequential(
            nn.Conv2d(128, 256, 3, padding = 1),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(256),
            nn.MaxPool2d(2)
            
        )
                
        #FC Layer
        self.drop = nn.Dropout2d(0.5)
        self.fc1 = nn.Linear(256*7*7, 512)
        self.fc2 = nn.Linear(512, len(images['train_images_100'].classes))
        
    #Forward function
    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.layer5(x)
                
        x = self.drop(x)
        
        x = self.fc1(x)
        x = self.drop(x)
        x = self.fc2(x)
        
        return x

In [122]:
#Instantiating a CNN
model = CNN()
if torch.cuda.is_available():
    gc.collect()
    
    torch.cuda.empty_cache()
    
    torch.cuda.CUDA_LAUNCH_BLOCKING=1
    
    model.cuda()

In [123]:
#Loss Function    
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [124]:
def get_accuracy(logit, target, batch_size):
    corrects = (torch.max(logit, 1)[1].view(target.size()).data == target.data).sum()
    accuracy = 100.0 * corrects/batch_size
    return accuracy.item()

In [125]:
num_epochs = 10
BATCH_SIZE = 32

In [126]:
#Training the model        
for epoch in range(num_epochs):
    train_loss = 0.0
    train_acc = 0.0
    
    model.train()
    for i, (images, labels) in enumerate (loader['train_images_100']):
        if torch.cuda.is_available():
            images, labels = images.cuda(), labels.cuda()
            
        logits = model(images)
        loss = criterion(logits, labels)
        optimizer.zero_grad()
        loss.backward()
        
        optimizer.step()
        
        train_loss += loss.detach().item()
        train_acc += get_accuracy(logits, labels, BATCH_SIZE)
        
    model.eval()
        
    print('Epoch: %d | Loss: %.4f | Train Accuracy: %.2f' \
        %(epoch, train_loss / i, train_acc/i))
        
    test_acc = 0.0
    for i, (images, labels) in enumerate (loader['test_images_100']):
        if torch.cuda.is_available():
            images, labels = images.cuda(), labels.cuda()
        
        outputs = model(images)
        test_acc += get_accuracy(outputs, labels, BATCH_SIZE)
    
    print('Test Accuracy: %.2f'%( test_acc/i))

RuntimeError: CUDA out of memory. Tried to allocate 62.00 MiB (GPU 0; 4.00 GiB total capacity; 470.93 MiB already allocated; 0 bytes free; 480.00 MiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF